In [100]:
import derive
reload(derive)
import sympy as sm
import tensorflow as tf

fun = lambda x: (3 * x[0] + x[1]) ** 2
point = [0.1, 0.3]

grad = derive.grad(fun)
print grad(point)
hess = derive.hess(fun)
print hess(point)

x, y = sm.symbols('x, y')
fun = (3 * x + y) ** 2
grad = derive.grad(fun)
print grad(point)
hess = derive.hess(fun)
print hess(point)

import numpy as np
x = tf.Variable(0, dtype = tf.float32)
y = tf.Variable(0, dtype = tf.float32)
fun = 3 * x + y
hess = derive.hess(fun)
# print hess(point)

[ 3.6       1.200006]
[[ 17.99994875   5.99995054]
 [  5.99995054   2.00008066]]
[3.5999999999999996, 1.2000000000000002]


SympifyError: Sympify of expression 'could not parse u'<function eval at 0x7f2e8eefeaa0>'' failed, because of exception being raised:
SyntaxError: invalid syntax (<string>, line 1)

In [72]:
import theano
import theano.tensor as T

# http://deeplearning.net/software/theano/tutorial/gradients.html
    
x = T.dvector('x')
y = x ** 2
cost = y.sum()
gy = T.grad(cost, x)
H, updates = theano.scan(
    lambda i, gy, x: T.grad(gy[i], x),
    sequences = T.arange(gy.shape[0]),
    non_sequences = [gy, x]
)
f = theano.function([x], H, updates = updates)
print f([4, 4])

[[ 2.  0.]
 [ 0.  2.]]


In [76]:
import theano
import theano.tensor as T

# http://stackoverflow.com/questions/30500332/how-to-use-theano-gradient-hessian-example-needed

x = T.dvector('x')
y = T.dvector('y')
input = [x, y]
cost = T.sum(x ** 2 + y ** 2)

f = theano.gradient.hessian(cost, wrt = input)
h = theano.function(input, f)

x = [1, 2]
y = [1, 2]
print h(x, y)

[array([[ 2.,  0.],
       [ 0.,  2.]]), array([[ 2.,  0.],
       [ 0.,  2.]])]


In [1]:
import scipy.optimize as opt
from converge import Converge
from numpy.linalg import inv

def minimize(f, x, g, H, epsilon = 0.0000001, maxsteps = 100):
    converge = Converge(f, epsilon, maxsteps)
    
    while not converge.done(x):
        d = -inv(H(x)).dot(g(x))
        x += d * opt.minimize(lambda eta: f(x + eta * d), 1).x
    return x

In [ ]:
from scipy import stats
import scipy.optimize as opt

pdf = stats.multivariate_normal((23, 7), 4).pdf
fun = lambda x: -pdf(x)
x = [10, 10]
print minimize(fun, x, derive.grad(fun), derive.hess(fun)) 
print opt.minimize(fun, x).x